In [21]:
import cv2 as cv2
import os
from skimage import io, measure
import numpy as np
import matplotlib.pyplot as plt
import statistics

In [22]:
def read_meteor(path, in_folder="data/meteor/", out_folder=None):
    mask = io.imread(in_folder + path + "_mask.png")

    img = io.imread(in_folder + path + ".tif")
    
    background = img[mask == 0]
    median = statistics.median(background)
    img[mask == 0] = 0
    percentile = np.percentile(img[mask != 0], 95)
    img[img > percentile] = percentile
    img = (np.abs(img - median) / (percentile - median) * 255).astype(np.uint8)

    if out_folder is not None:
        plt.imsave(out_folder + path + ".png", img, cmap='gray')

    return img, mask

In [23]:
def detect_meteor(img, path=None, closing_factor=5, out_folder=None):
    
    threshold=50
    while True:
        bin_img = (img > threshold)
        if len(bin_img[bin_img]) > 5:
            break
        threshold -= 5

    img = bin_img.astype(np.uint8)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (closing_factor, closing_factor))
    closed_img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    lines = cv2.HoughLinesP(closed_img, rho=1, theta=np.pi/180, threshold=10)

    if out_folder is not None:
        plt.imsave(out_folder + path + "_closed.png", closed_img, cmap='gray')

    if lines is None:
        if path is not None:
            print('Lines not found in "' + path + '".', len(img[img != 0]))
        else:
            print("Lines not found.")
        return None, closed_img
    
    l = len(lines)
    lines = lines.squeeze()
    if l == 1:
        lines = np.array([lines])

    longest_line = max(lines, key=lambda line: (line[2] - line[0])**2 + (line[3] - line[1])**2)

    if out_folder is not None:
        x1, y1, x2, y2 = longest_line
        img_color = cv2.cvtColor(img * 255, cv2.COLOR_GRAY2BGR)
        cv2.line(img_color, (x1, y1), (x2, y2), (0, 0, 255), 2)
        plt.imsave(out_folder + path + "_hough.png", img_color)

    return longest_line, closed_img

In [24]:
def detect_meteor_2(img, path=None, closing_factor=5, out_folder=None):
    
    threshold=50
    while True:
        bin_img = (img > threshold)
        if len(bin_img[bin_img]) > 5:
            break
        threshold -= 5

    img = bin_img.astype(np.uint8)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (closing_factor, closing_factor))
    closed_img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    properties = measure.regionprops(closed_img)

    if out_folder is not None:
        plt.imsave(out_folder + path + "_closed.png", closed_img, cmap='gray')

    if len(properties) == 0:
        if path is not None:
            print(f'Lines not found in "{path}".', len(img[img != 0]))
        else:
            print("Lines not found.")
        return None, closed_img
    
    longest_line = None
    max_length = 0

    for prop in properties:
        cy, cx = prop.centroid
        orientation = prop.orientation + np.pi / 2
        length = prop.major_axis_length

        if length > max_length:
            max_length = length
            length = length / 2
            longest_line = (cx - length * np.cos(orientation), cy + length * np.sin(orientation),
                            cx + length * np.cos(orientation), cy - length * np.sin(orientation))

    if out_folder is not None:
        img_color = cv2.cvtColor(img * 255, cv2.COLOR_GRAY2BGR)
        x1, y1, x2, y2 = longest_line
        cv2.line(img_color, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        plt.imsave(out_folder + path + "_regionprops.png", img_color)

    return longest_line, closed_img

In [25]:
def rotate_meteor(imgs, line, out_path=None):
    x1, y1, x2, y2 = line
    dx = x2 - x1
    dy = y2 - y1
    angle_radians = np.arctan2(dy, dx)
    angle_degrees = np.degrees(angle_radians)

    rotated_imgs = []
    for img in imgs:
        rows, cols = img.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle_degrees, 1)
        rotated_img = cv2.warpAffine(img, rotation_matrix, (cols, rows))
        rotated_imgs.append(rotated_img)
    
    if out_path is not None:
        for i in range(len(rotated_imgs)):
            rotated_img = rotated_imgs[i]
            plt.imsave(out_path + "_" + str(i) + ".png", rotated_img, cmap="gray")
    
    return rotated_imgs

In [26]:
def crop_meteor(img, mask, out_path=None):    
    x, y, w, h = cv2.boundingRect(mask)
    img = img[y:y+h, x:x+w]

    if out_path is not None:
        plt.imsave(out_path + ".png", img, cmap='gray')

    return img

In [27]:
def spectral_from_meteor(img, out_path=None):
    dft = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft = np.fft.fftshift(dft)

    magnitude_spectral_full = cv2.magnitude(dft[:,:,0], dft[:,:,1])

    center_index = int(magnitude_spectral_full.shape[0] / 2)
    magnitude_1d = magnitude_spectral_full[center_index,:]
    
    if out_path is not None:
        plt.plot(magnitude_1d)
        plt.savefig(out_path + "_simple.png")
        plt.close()

    return magnitude_1d

In [28]:
def freq_dist_to_pixel_dist(freq_dist, magnitude_1d):
    pixel_dist = 1/ ( freq_dist / magnitude_1d.shape[0] )
    return pixel_dist

def pixel_dist_to_pixel_speed(pixel_dist, refresh_duration=20):
    meteor_speed_ms = pixel_dist / refresh_duration
    meteor_speed_s = meteor_speed_ms * 1000
    return meteor_speed_s

def pixel_speed_to_real_speed(meteor_speed_s, dist_per_pixel=100):
    real_speed = meteor_speed_s * dist_per_pixel
    return real_speed

def meteor_speed(magnitude_1d, path=None, out_folder=None):
    magnitude_1d_begin = np.insert(magnitude_1d, 0, magnitude_1d[0]) # 0 begin
    magnitude_1d_end   = np.append(magnitude_1d, magnitude_1d[-1]) # 0 end

    derivate = magnitude_1d_end - magnitude_1d_begin
    supremums = np.all([derivate[:-1] > 0, derivate[1:] < 0], axis=0)

    x_supremums = np.where(supremums)[0]
    y_supremums = np.array(magnitude_1d[supremums])

    indexes = np.flip(np.argsort(y_supremums))

    band_width = len(indexes) * 0.2

    center_index = x_supremums[indexes[0]]

    max_index_i = 1
    while True:
        if len(x_supremums) <= max_index_i:
            if path is not None:
                print(max_index_i + 1, 'Supremums not found in "' + path + '".')
            else:
                print(max_index_i + 1, "2 Supremums not found.")

            if out_folder is not None:
                plt.plot(magnitude_1d)
                plt.plot(x_supremums[indexes[:max_index_i]], y_supremums[indexes[:max_index_i]], 'r*')
                plt.plot(x_supremums[indexes[0]] - band_width, 0, 'g+')
                plt.plot(x_supremums[indexes[0]] + band_width, 0, 'g+')
                plt.savefig(out_folder + path + "_supremums.png")
                plt.close()

            return None

        max_index = x_supremums[indexes[max_index_i]]
        if abs(max_index - center_index) > band_width:
            break

        max_index_i += 1

    if out_folder is not None:
        plt.plot(magnitude_1d)
        plt.plot(x_supremums[indexes[:max_index_i]], y_supremums[indexes[:max_index_i]], 'r*')
        plt.plot(x_supremums[indexes[0]] - band_width, 0, 'g+')
        plt.plot(x_supremums[indexes[0]] + band_width, 0, 'g+')
        plt.plot(x_supremums[indexes[max_index_i:max_index_i+2]], y_supremums[indexes[max_index_i:max_index_i+2]], 'g*')
        plt.savefig(out_folder + path + "_supremums.png")
        plt.close()

    dist_freq = abs(center_index - max_index)

    dist_pixel = freq_dist_to_pixel_dist(dist_freq, magnitude_1d)

    pixel_speed = pixel_dist_to_pixel_speed(dist_pixel)
    real_speed = pixel_speed_to_real_speed(pixel_speed)

    return real_speed

In [29]:
folder_path = 'data/meteor'

paths = []
for filename in os.listdir(folder_path):
    if filename.endswith('.tif'):
        file_path = os.path.join(folder_path, filename)
        paths.append(file_path.split('.')[0].split('/')[-1])

print(len(paths), paths[:1])

867 ['20160811_000103_487_700_766_1250_1358']


In [30]:
# GLOBAL FRAMEWORK
errors = 0

f = open("data/speeds.csv", 'w')
f.write("name;speed;error\n")

# paths = ['20140711_025814_533_90_300_485_560', '20160811_004211_021_137_446_406_883', '20150810_010909_186_264_388_399_641']

for path in paths:
    f.write(path + ';')

    img, mask = read_meteor(path, out_folder="data/cleaned/")
    line, closed = detect_meteor_2(img, path, out_folder="data/lines/")
    if line is not None:
        img, mask = rotate_meteor([img, mask], line, out_path="data/rotations/" + path)
        img = crop_meteor(img, mask, out_path="data/cropped/" + path)
        magnitude = spectral_from_meteor(img, out_path="data/spectral/" + path)
        speed = meteor_speed(magnitude, path, out_folder="data/spectral/")
        if speed is not None:
            f.write(str(speed) + ";\n")
        else:
            errors += 1
            f.write("NaN;supremums not found\n")

    else:
        errors += 1
        f.write("NaN;lines not found\n")

f.close()

print(errors, "images cannot give a result.")

2 Supremums not found in "20141027_015812_464_96_148_704_757".
1 images cannot give a result.
